In [1]:
import sys
import os
sys.path.append('..')
import edlib
import numpy as np
from collections import Counter, defaultdict
import operator
from string import ascii_uppercase
from itertools import groupby
from copy import deepcopy


from lrd_parser import LRD_Report
from utils.bio import hamming_distance, identity_shift, OverlapAlignment, compress_homopolymer
from utils.os_utils import smart_makedirs
import networkx as nx
from debruijn_graph import DeBruijnGraph, iterative_graph, get_frequent_kmers, get_all_kmers

import matplotlib
%matplotlib inline 
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

from ndex2.nice_cx_network import NiceCXNetwork
import ndex2.client as nc
import ndex2

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'lrd_parser'

# Read and correct gaps

In [ ]:
def get_ngaps(strings):
    ngaps = 0
    for r_id, string in strings.items():
        r_ngaps = Counter(string)['=']
        if r_ngaps >= 5:
            mstring = string.lower()
            mstring = mstring.replace('=', '|')
            # print(r_id)
            # print(mstring)
            # print("")
        ngaps += r_ngaps
    return ngaps

def simple_stats(monomer_strings):
    monomer_string_lens = [len(monomer_string) for monomer_string in monomer_strings.values()]
    print(f'Len: Mean = {np.mean(monomer_string_lens)}, ' +
          f'Min = {np.min(monomer_string_lens)}, ' +
          f'Max = {np.max(monomer_string_lens)}, ' +
          f'Total = {np.sum(monomer_string_lens)}')
    ngaps = get_ngaps(monomer_strings)
    print(f'#gaps = {ngaps}, %gaps = {ngaps / np.sum(monomer_string_lens)}')

In [ ]:
lrd_report_fn = '/Poppy/abzikadze/centroFlye/centroFlye_repo/experiments/20191023/lrd_d6z1_rel3_Karen/decomposition.tsv'
monomers_fn = '/Poppy/abzikadze/tandem_flye/data/human/isolated_centromeres/extracted_HORs/CEN6/monomers/inferred_monomers_single.fa'


lrd_report = LRD_Report(lrd_report_fn=lrd_report_fn, monomers_fn=monomers_fn)

In [ ]:
monomer_strings = {r_id: record.string for r_id, record in lrd_report.records.items()}
len(monomer_strings)

In [ ]:
simple_stats(monomer_strings)

In [ ]:
def filter_strings(monomer_strings, max_gap=0.05, max_lowercase=0.01):
    filtered_strings = {}
    for r_id, string in monomer_strings.items():
        ngaps = Counter(string)['=']
        lowercase = [s.islower() for s in string]
        if np.mean(lowercase) > max_lowercase:
            continue
        if ngaps / len(string) <= max_gap:
            filtered_strings[r_id] = string
    return filtered_strings

In [ ]:
monomer_strings = filter_strings(monomer_strings)
len(monomer_strings)

In [ ]:
simple_stats(monomer_strings)

In [ ]:
def get_ma(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

def trim_read(monomer_string, max_gap, ma_window):
    is_gap = [c == '=' for c in monomer_string]
    ma = get_ma(is_gap, N=ma_window)
    l = 0
    while l < len(ma) and ma[l] > max_gap:
        l += 1
    r = len(ma) - 1
    while r >= 0 and ma[r] > max_gap:
        r -= 1
    trimmed_read = monomer_string[l:r+1+ma_window]
    trimmed_read = trimmed_read.strip('=')
    trimmed_length = l + len(ma)-(r+1+ma_window)
    return trimmed_read, trimmed_length

def trim_reads(monomer_strings, max_gap=0.2, ma_window=30):
    trimmed_reads = {}
    for r_id, monomer_string in monomer_strings.items():
        trimmed_read, trimmed_length = \
            trim_read(monomer_string, max_gap=max_gap, ma_window=ma_window)
        trimmed_reads[r_id] = trimmed_read
    return trimmed_reads

In [ ]:
monomer_strings = trim_reads(monomer_strings)
len(monomer_strings)

In [ ]:
simple_stats(monomer_strings)

In [ ]:
frequent_kmers, frequent_kmers_read_pos = get_frequent_kmers(monomer_strings, k=3, min_mult=1000)
db = DeBruijnGraph(k=3)
db.add_kmers(frequent_kmers, coverage=frequent_kmers)

#hors, _ = db.get_contigs()
print(db.get_contigs())
nx.drawing.nx_pydot.write_dot(db.graph, 'db.dot')


def correct_gaps(monomer_strings, hors, max_gap=0.3, nhor=5):
    corrected_strings = {}
    for r_id, monomer_string in monomer_strings.items():
            corrected_string = list(monomer_string)
            for single_hor in hors:
                for i_nhor in range(1, nhor+1):
                    hor = single_hor * i_nhor
                    hor_len = len(hor)
                    for i in range(len(monomer_string)-hor_len+1):
                        kmer = monomer_string[i:i+hor_len]
                        gap_cnt = Counter(kmer)['=']
                        if gap_cnt == 0 or gap_cnt / hor_len > max_gap:
                            continue
                        hd, _ = hamming_distance(kmer, hor, match_char=set('='))
                        if hd == 0:
                            # print(hor)
                            # print(kmer)
                            # print("")
                            corrected_string[i:i+hor_len] = list(hor)
                        
            corrected_strings[r_id] = ''.join(corrected_string)
    return corrected_strings

# monomer_strings = correct_gaps(monomer_strings, hors)
# ngaps = get_ngaps(monomer_strings)
# print(ngaps)

In [ ]:
simple_stats(monomer_strings)

In [ ]:
def get_read_coverage(frequent_kmers_read_pos, monomer_strings, k):
    coverage = {}
    for r_id, string in monomer_strings.items():
        coverage[r_id] = [0] * (len(string) + 1)
    for pairs in frequent_kmers_read_pos.values():
        for r_id, pos in pairs:
            coverage[r_id][pos] += 1
            coverage[r_id][pos+k] -= 1
    for r_id in coverage:
        coverage[r_id] = np.cumsum(coverage[r_id])
        coverage[r_id] = coverage[r_id][:-1]
    return coverage

def find_zero_cov(coverage):
    all_zero_cov = {}
    for r_id in coverage:
        zero_cov_flatten = np.where(coverage[r_id] == 0)[0]
        if len(zero_cov_flatten) == 0:
            all_zero_cov[r_id] = []
            continue
        zero_cov = []
        zero_cov.append([zero_cov_flatten[0]])
        for pos in zero_cov_flatten[1:]:
            if pos == zero_cov[-1][-1] + 1:
                zero_cov[-1].append(pos)
            else:
                zero_cov.append([pos])
        
        all_zero_cov[r_id] = [(x[0], x[-1]) for x in zero_cov]
        
    return all_zero_cov

def find_path_debr(zero_cov, read_seq, r_id, k, db,
                   max_len=1000, min_len=1, min_overlap=3, max_overlap=30, max_dist=1):
    k -= 1
    results = {}
    corrected_seq = list(read_seq)
    for st, en in zero_cov[::-1]:
        if st < k or en > len(read_seq) - 1 - k or en-st+1 > max_len or en-st+1 < min_len:
            # results[(st, en)] = '-'
            continue
        # print(r_id, st, en)
        st_kmer, en_kmer = read_seq[st-k:st], read_seq[en+1:en+1+k]
        # assert st_kmer in frequent_kmers
        # assert en_kmer in frequent_kmers
        # assert read_seq[st-k+1:st+1] not in frequent_kmers
        # assert read_seq[en:en+k] not in frequent_kmers
        # assert st_kmer in db.graph.nodes

        u = st_kmer
        kmers = [u]
        while u != en_kmer and len(kmers) < 2*k:
            u_node = db.node_mapping[u]
            out_edges = list(db.graph.out_edges(u_node))
            # print(list(db.graph.nodes(data=True))[0])
            # print(u, len(db.graph.in_edges(u_node)), len(out_edges))
            # print(out_edges)
            if len(out_edges) == 1:
                edge = out_edges[0]
                assert edge[0] == u_node
                u_node = edge[1]
                u = db.rev_node_mapping[u_node]
                kmers.append(u)
            else:
                break
            
        if len(kmers) < min_overlap + 1:
            # results[(st, en)] = '-'
            # print(kmers)
            continue

        # print(len(kmers))
        extension = [kmer[-1] for kmer in kmers[1:]]
        extension = ''.join(extension)
        read_segment = read_seq[en+1:en+len(extension)]
        # print(extension)
        # print(read_segment)
        
        ident = identity_shift(extension[:max_overlap],
                               read_segment[:max_overlap],
                               min_overlap=min_overlap,
                               match_char=set('='))
        # print(ident)
        if ident['id'] == 1 and ident['shift'] is not None:
            # print(ident)
            correction = extension[:ident['shift']]
            if abs(len(correction) - (en-st+1)) > 10:
                # results[(st, en)] = '-'
                continue
            print(read_seq[st:en+1], read_seq[st-5:en+6])
            print(correction, len(correction))
            print("")
            results[(st, en)] = (read_seq[st:en+1], correction)
            corrected_seq[st:en+1] = list(correction)
    corrected_seq = ''.join(corrected_seq)
    return results, corrected_seq


def correct_seq(monomer_strings, k, min_mult=10):
    frequent_kmers, frequent_kmers_read_pos = get_frequent_kmers(monomer_strings, k=k, min_mult=min_mult)
    db = DeBruijnGraph(k=k)
    db.add_kmers(frequent_kmers, coverage=frequent_kmers)
    coverage = get_read_coverage(frequent_kmers_read_pos=frequent_kmers_read_pos,
                                 monomer_strings=monomer_strings,
                                 k=k)
    zero_cov = find_zero_cov(coverage=coverage)
    
    all_corrections = {}
    corrected_seqs = {}
    for r_id in zero_cov:
        all_corrections[r_id], corrected_seqs[r_id] = find_path_debr(zero_cov[r_id],
                                                              monomer_strings[r_id],
                                                              r_id=r_id,
                                                              k=k,
                                                              db=db)
    return all_corrections, corrected_seqs


def correct_reads(monomer_strings, min_k=10, max_k=200, niter=1):
    corrected_seqs = monomer_strings
    for k in range(min_k, max_k):
        for i in range(niter):
            print(k, i)
            all_corrections, corrected_seqs = correct_seq(corrected_seqs, k=k)
            for r_id, corrections in all_corrections.items():
                if len(corrections):
                    print(r_id, corrections)
    return corrected_seqs

In [ ]:
# monomer_strings = correct_reads(monomer_strings)

In [ ]:
# ngaps = get_ngaps(monomer_strings)
# print(ngaps)

# Iterative construction

In [ ]:
min_k, max_k = 10, 400
contigs, dbs, all_frequent_kmers, all_frequent_kmers_read_pos = \
    iterative_graph(monomer_strings, min_k=min_k, max_k=max_k,
                    outdir='/Poppy/abzikadze/centroFlye/centroFlye_repo/experiments/20191115/db_wi_error_corr')

In [ ]:
# After error correction i could get to max_k = 634
# min_k, max_k = 10, 634
# contigs, dbs = iterative_graph(monomer_strings, min_k=min_k, max_k=max_k)

In [ ]:
# db = dbs[max(dbs.keys())]
# db = dbs[634]
# db = dbs2[500]
# db = dbs[650]


# assert nx.number_weakly_connected_components(db.graph) == 1

# Mapping of reads

In [ ]:
len(contigs)

In [ ]:
contig_lens = sorted(len(contig) for contig in contigs)
print(contig_lens)

In [ ]:
edges, coverage = db.get_edges()

In [ ]:
edge_lens = sorted(len(edge) for edge in edges)
print(edge_lens)

In [ ]:
nx.drawing.nx_pydot.write_dot(db.graph, 'db.dot')

In [ ]:
#nice_cx_debr_graph = ndex2.create_nice_cx_from_networkx(db.graph)

#nice_cx_debr_graph.upload_to(server='public.ndexbio.org', username = 'seryrzu',
#                             password = 'Kxoq)V?Z]vrgt87x*XO,:we)U&RwEEG!')

In [ ]:
def map_reads(monomer_strings, db, db_index=None):
    if db_index is None:
        db_index = db.index_edges()
    mapping = {}
    db_edges = list(db.graph.edges(keys=True))
    for r_id, string in monomer_strings.items():
        split_strings = list(filter(lambda string: len(string), string.split('=')))
        split_lens = [0] + [len(split_string) for split_string in split_strings]
        cum_split_lens = np.cumsum(split_lens)
        read_coords = []
        for split_ind, split_string in enumerate(split_strings):
            for i in range(len(split_string)-db.k+1):
                kmer = split_string[i:i+db.k]
                if kmer in db_index[len(kmer)]:
                    read_coords.append(db_index[len(kmer)][kmer])
        
        path = [x[0] for x in read_coords]
        path = [x[0] for x in groupby(path)]
        path = [db_edges[edge_ind] for edge_ind in path]
        
        valid_path = True
        for e1, e2 in zip(path[:-1], path[1:]):
            if e1[1] != e2[0]:
                valid_path = False
                break
        if len(read_coords):
            mapping[r_id] = (read_coords[0], read_coords[-1], valid_path, path)
        else:
            mapping[r_id] = None
    return mapping

mappings = map_reads(monomer_strings, db)

print(np.mean([read_mapping is not None for read_mapping in mappings.values()]))
print(np.mean([read_mapping[2] for read_mapping in mappings.values() \
               if read_mapping is not None]))
print(np.sum([read_mapping[2] for read_mapping in mappings.values() \
               if read_mapping is not None]))

for r_id, read_mapping in mappings.items():
    if read_mapping is not None and read_mapping[2]:
        print(r_id, len(monomer_strings[r_id]), len(monomer_strings[r_id]) * 171,
              len(db.get_path(read_mapping[-1])), read_mapping)



In [ ]:
def filter_mappings(mappings, db):
    filtered_mappings = {}
    contigs, contig_paths = db.get_contigs()
    for r_id, read_mapping in mappings.items():
        if read_mapping is None or not read_mapping[-2]:
            continue
        read_mapping = read_mapping[-1]
        rm_len = len(read_mapping)
        if rm_len == 1:
            continue
        in_contig = False
        for path in contig_paths:
            path = list(path)
            for i in range(len(path)-rm_len+1):
                subpath = path[i:i+rm_len]
                # print(subpath)
                # print(read_mapping)
                if subpath == read_mapping:
                    in_contig = True
                    break
            if in_contig:
                break
        if not in_contig:
            filtered_mappings[r_id] = read_mapping
            # print(r_id, read_mapping)
        else:
            pass
            # print(r_id, read_mapping)
    return filtered_mappings

In [ ]:
scaffolding_reads = filter_mappings(mappings, db)

In [ ]:
len(scaffolding_reads)

In [ ]:
def get_scaffold_links(scaffolding_reads, db):
    scaffolds = defaultdict(list)
    contigs, contig_paths = db.get_contigs()
    for r_id, scaffolding_read in scaffolding_reads.items():
        best_lo, best_lo_len, best_lo_shift = None, 0, 0
        best_ro, best_ro_len, best_ro_shift = None, 0, 0
        
        for i, path in enumerate(contig_paths):
            l_overlap = identity_shift(path, scaffolding_read, min_overlap=1)
            r_overlap = identity_shift(scaffolding_read, path, min_overlap=1)
            if l_overlap['id'] == 1 and len(l_overlap['alt_shifts']) == 0 \
                    and l_overlap['len'] >= best_lo_len \
                    and l_overlap['shift'] + len(scaffolding_read) > len(path) \
                    and l_overlap['shift'] >= 1:
                best_lo = list(path)
                best_lo_len = l_overlap['len']
                best_lo_shift = l_overlap['shift']
            
            if r_overlap['id'] == 1 and len(r_overlap['alt_shifts']) == 0 \
                    and r_overlap['len'] >= best_ro_len \
                    and r_overlap['shift'] + len(path) > len(scaffolding_read) \
                    and r_overlap['shift'] >= 1:
                best_ro = list(path)
                best_ro_len = r_overlap['len']
                best_ro_shift = r_overlap['shift']
            
        if best_lo is not None and best_ro is not None:
            scaffold = best_lo[:best_lo_shift] + scaffolding_read[:best_ro_shift] + best_ro
            scaffold = tuple(scaffold)
            scaffolds[scaffold].append(r_id)
#             print('left')
#             print(r_id)
#             print('read', scaffolding_read)
#             print('contig', best_lo)
#             print("")
#             print('right')
#             print(r_id)
#             print('read', scaffolding_read)
#             print('contig', best_ro)
#             print("")
#             print("scaffold:", scaffold)
#             print(len(db.get_path(scaffold)))
#             print("")
    
    return scaffolds
            

In [ ]:
scaffolds = get_scaffold_links(scaffolding_reads, db)

In [ ]:
for scaffold in scaffolds:
    print(scaffold, len(db.get_path(scaffold)))

# Manual scaffolding of long edges in graph with k=400

In [ ]:
from string import ascii_uppercase

man_assembly_fn = "/Poppy/abzikadze/centroFlye/centroFlye_repo/data/D6Z1/CEN6_ManVERSION3.tsv"

units = []
with open(man_assembly_fn) as f:
    for line in f:
        line = line.strip().split('\t')
        st, en = int(line[-2]), int(line[-1])
        if en < 17:
            en -= 1
        units.append((st, en))

from string import ascii_uppercase
from itertools import groupby

def monomers2units(s, unit_len=18):
    units = []
    i = 0
    while i < len(s):
        u_st = i
        j = 0
        shift = ascii_uppercase.find(s[i])
        if shift == -1:
            i += 1
            units.append('=')
            continue
        while i + j < len(s) and ascii_uppercase[j+shift] == s[i+j]:
            j += 1
        if shift == 0 and j == unit_len:
            units.append('Full')
        else:
            units.append((shift, shift+j-1))
        i += j
    compressed_units = []
    for key, group in groupby(units):
        compressed_units.append((key, len(list(group))))
    return compressed_units
        
def units2monomers(units):
    monomers = []
    for (s, e) in units:
        monomers.append(ascii_uppercase[s:e+1])
    monomers = ''.join(monomers)
    return monomers

ref_monomers = units2monomers(units)

def get_cov(ref_monomers, seqs, max_ed=10000):
    coverage = [0] * (len(ref_monomers)+1)
    for r_id, seq in seqs.items():
        alignment = edlib.align(seq,
                                ref_monomers,
                                mode='HW',
                                task='locations',
                                k=max_ed)
        print(r_id, alignment, np.diff(alignment['locations']))
        loc = alignment['locations']
        if loc is None or len(loc) != 1:
            continue
        else:
            loc = loc[0]
        coverage[loc[0]] += 1
        coverage[loc[1]+1] -= 1
    coverage = np.cumsum(coverage)
    return coverage

In [ ]:
db = dbs[400]

In [ ]:
def get_long_edges(db, min_len=1000, max_cov=30):
    edges = {}
    for edge in db.graph.edges(data=True, keys=True):
        edge_len = len(edge[-1]['edge_kmer'])
        edge_cov = np.median(edge[-1]['coverages'])
        if edge_len >= min_len and edge_cov <= max_cov:
            print(edge[:-1], edge_len, np.median(edge_cov))
            edges[edge[:-1]] = edge[-1]['edge_kmer']
    return edges

In [ ]:
long_edges = get_long_edges(db)
len(long_edges)

In [ ]:
cov = get_cov(ref_monomers, long_edges)
plt.plot(cov)

In [ ]:
long_reads = {k: v for k, v in monomer_strings.items() if len(v) >= 1000}
len(long_reads)

In [ ]:
cov = get_cov(ref_monomers, long_reads)
plt.plot(cov)

In [ ]:
def connect_edges(edges, reads, min_overlap=300, min_id=0.97):
    connections = defaultdict(list)
    for r_id, read in reads.items():
        b_pref_al, b_pref_overlap, b_pref_edge = None, 0, None
        b_suf_al, b_suf_overlap, b_suf_edge = None, 0, None
        
        # print("\n!!!!!\n")
        for e_id, edge in edges.items():
            er = identity_shift(edge, read, min_overlap, match_char=set('=?'))
            re = identity_shift(read, edge, min_overlap, match_char=set('=?'))
            if er['shift'] != None and len(er['alt_shifts']) == 0 and er['id'] > min_id \
                    and er['len'] > b_pref_overlap:
                b_pref_al = er
                b_pref_overlap = b_pref_overlap
                b_pref_edge = e_id
            
            if re['shift'] != None and len(re['alt_shifts']) == 0 and re['id'] > min_id \
                    and re['len'] > b_suf_overlap:
                b_suf_al = re
                b_suf_overlap = b_suf_overlap
                b_suf_edge = e_id
        
        if b_suf_edge is not None and b_pref_edge is not None:
            print(r_id)
            print(b_pref_edge, b_pref_al)
            print(b_suf_edge, b_suf_al)
            dist = len(read) - b_pref_al['len'] - b_suf_al['len']
            print(dist)
            connections[(b_pref_edge, b_suf_edge)].append(
                (r_id, b_pref_al['len'], b_pref_al['id'], dist, b_suf_al['len'], b_suf_al['id'])
            )
    return connections

In [ ]:
connections = connect_edges(long_edges, monomer_strings)

In [ ]:
cov = get_cov(ref_monomers, long_edges)
plt.plot(cov)

In [ ]:
connections

In [ ]:
for edge_pair, read_list in connections.items():
    dist_cnt = Counter(x[3] for x in read_list)
    mc = dist_cnt.most_common(1)
    if mc[0][1] >= 2:
        print(edge_pair)
        for r_id, pref_ov_len, pref_ov_id, dist, suf_ov_len, suf_ov_id in read_list:
            print(f'{r_id} {pref_ov_len:4} {pref_ov_id:5.2}, {dist:4}, {suf_ov_len:4}, {suf_ov_id:5.2}')
        print("")

In [ ]:
def map_reads(monomer_strings, db, db_index=None):
    if db_index is None:
        db_index = db.index_edges()
    mapping = {}
    db_edges = list(db.graph.edges(keys=True))
    for r_id, string in monomer_strings.items():
        split_strings = list(filter(lambda string: len(string), string.split('=')))
        split_lens = [0] + [len(split_string) for split_string in split_strings]
        cum_split_lens = np.cumsum(split_lens)
        read_coords = []
        for split_ind, split_string in enumerate(split_strings):
            for i in range(len(split_string)-db.k+1):
                kmer = split_string[i:i+db.k]
                if kmer in db_index[len(kmer)]:
                    read_coords.append(db_index[len(kmer)][kmer])
        
        path = [x[0] for x in read_coords]
        path = [x[0] for x in groupby(path)]
        path = [db_edges[edge_ind] for edge_ind in path]
        
        valid_path = True
        for e1, e2 in zip(path[:-1], path[1:]):
            if e1[1] != e2[0]:
                valid_path = False
                break
        if len(read_coords):
            mapping[r_id] = (read_coords[0], read_coords[-1], valid_path, path)
        else:
            mapping[r_id] = None
    return mapping

mappings = map_reads(monomer_strings, db)

print(np.mean([read_mapping is not None for read_mapping in mappings.values()]))
print(np.mean([read_mapping[2] for read_mapping in mappings.values() \
               if read_mapping is not None]))
print(np.sum([read_mapping[2] for read_mapping in mappings.values() \
               if read_mapping is not None]))

for r_id, read_mapping in mappings.items():
    if read_mapping is not None and read_mapping[2]:
        print(r_id, len(monomer_strings[r_id]), len(monomer_strings[r_id]) * 171,
              len(db.get_path(read_mapping[-1])), read_mapping)

## Connection between A  = (7, 9915, 0) and B = (5120, 5451, 0)

In [ ]:
# monomers2units(long_edges[(7, 9915, 0)]) # A in unit form
# monomers2units(long_edges[(5120, 5451, 0)]) # B in unit form

In [ ]:
# monomers2units(db.graph.get_edge_data(9915, 7, 0)['edge_kmer'])

In [ ]:
# monomers2units(db.graph.get_edge_data(14132, 5491, 0)['edge_kmer'])

In [ ]:
# monomers2units(monomer_strings['ab79a298-1f1a-44da-a174-6a52ae8abfcd'])

In [ ]:
# monomer_strings['0c50866b-f780-4551-a4bb-aa35a72f3f4d'][323:323+136]

In [ ]:
# monomer_strings['1b4bd09e-07f3-4455-b041-c4599b6fbe3d'][901:901+138]

In [ ]:
# get_paths_thru_complex_nodes(dbs[303], monomer_strings)

In [ ]:
# monomers2units(dbs[303].graph.get_edge_data(7, 8, 0)['edge_kmer'])

In [ ]:
# monomers2units(dbs[303].graph.get_edge_data(8, 7, 0)['edge_kmer'])

In [ ]:
# mappings = map_reads({'ab79a298-1f1a-44da-a174-6a52ae8abfcd': monomer_strings['ab79a298-1f1a-44da-a174-6a52ae8abfcd']},
#                     dbs[304])

In [ ]:
# mappings

In [ ]:
# dbs[303].rev_node_mapping[8]

In [ ]:
# dbs[303].rev_node_mapping[3199]

In [ ]:
# dbs[303].rev_node_mapping[7]

In [ ]:
# dbs[304].rev_node_mapping[7]

In [ ]:
# erroneous_kmer = dbs[304].graph.get_edge_data(6732, 7, 0)['edge_kmer']
# len(erroneous_kmer)

In [ ]:
# cnt_erroneous_kmer = 0
# for _ in monomer_strings.values():
#     __ = Counter([_[i:i+len(erroneous_kmer)] for i in range(len(_)-len(erroneous_kmer)+1)])
#     cnt_erroneous_kmer += __[erroneous_kmer]
# print(cnt_erroneous_kmer)

In [ ]:
# dbs[304].graph.get_edge_data(7, 13999, 0)['edge_kmer'][:303]

In [ ]:
# erroneous_kmer

In [ ]:
# dbs[303].graph.get_edge_data(8, 13964, 0)['edge_kmer'][:302]

In [2]:
# dbs[303].graph.get_edge_data(7, 8, 0)['edge_kmer']

In [3]:
# all_frequent_kmers_read_pos[304][dbs[304].graph.get_edge_data(6732, 7, 0)['edge_kmer']]

In [4]:
# for contig_path in dbs[303].get_contigs()[1]:
#     if dbs[303].get_path(contig_path) == dbs[303].get_contigs()[0][21]:
#        print(contig_path)
#         print(dbs[303].get_path(contig_path))
#         print(erroneous_kmer in dbs[303].get_path(contig_path))

In [5]:
# print(dbs[303].rev_node_mapping[7])

## Connection between B = (5120, 5451, 0) and C  = (4445, 10624, 0) 

In [6]:
mappings = map_reads({'aff6bdf9-4388-46e6-9aa5-af5c6c88258d': monomer_strings['aff6bdf9-4388-46e6-9aa5-af5c6c88258d']},
                     dbs[400])



NameError: name 'map_reads' is not defined

In [ ]:
mappings

In [ ]:
def code_hor(units):
    code_units = []
    i = 0
    while i < len(units) - 1:
        u1, u2 = units[i], units[i+1]
        if u1 == ((0, 1), 1) and u2 == ((5, 17), 1):
            code_units.append('M')
            i += 2
        else:
            code_units.append(u1)
            i += 1
    compressed_units = []
    for key, group in groupby(code_units):
        if len(list(group)) == 1:
            compressed_units.append(key)
        else:
            compressed_units.append((key[0], len(list(group))))
    return compressed_units

In [ ]:
monomers2units(long_edges[(4445, 10624, 0)]) # C in unit form

In [ ]:
mappings = map_reads({'f3a7eb72-23c5-4c60-97d8-be9c7cb6285b': monomer_strings['f3a7eb72-23c5-4c60-97d8-be9c7cb6285b']},
                     dbs[400])



In [ ]:
mappings

## Connection between C  = (4445, 10624, 0) and D = (942, 7050, 0)

In [ ]:
mappings = map_reads(monomer_strings,
                     dbs[400])



In [ ]:
monomers2units(long_edges[(942, 7050, 0)]) # D in unit form

In [ ]:
mappings['2d7e41d5-9657-4c4a-bafe-de2baf162d51']

In [ ]:
mappings['347cb721-6ad0-4df3-9bd2-4bc0a3c08c8d']

In [ ]:
mappings['459cabbd-30ad-4a2a-985a-5983b49e50ff']

In [ ]:
mappings['535aa52e-6acc-4f47-af7f-e728e2b8c0bc']

In [ ]:
dbs[400].rev_node_mapping[13393]

In [ ]:
dbs[400].rev_node_mapping[11008]

In [ ]:
for i, (c1, c2) in enumerate(zip(dbs[400].rev_node_mapping[13393], dbs[400].rev_node_mapping[11008])):
    if c1 == c2 :
        print(i, c1.lower(), c2.lower())
    else:
        print(i, c1, c2, '!')

In [ ]:
all_frequent_kmers_read_pos[399][dbs[400].rev_node_mapping[11008]]

In [ ]:
all_frequent_kmers_read_pos[399][dbs[400].rev_node_mapping[13393]]

In [ ]:
mappings['9bf34c41-751f-4ef2-b150-b4e287b12eaa']

In [ ]:
mappings['9c6be407-612e-4e37-becb-34e62c64c7e2']

In [ ]:
mappings['b55aea96-cbe9-4c7b-9b0a-0501dd830ba8']

In [ ]:
mappings['d2f81138-e67b-4063-bdeb-3a4f5f9c31e0']

## Connection between D = (942, 7050, 0) and E  = (5120, 11585, 0) 

In [ ]:
mappings['86fdd7a8-71b5-46fa-90d2-5e6888ae2289']

In [ ]:
def get_edge_cov(mappings, db):
    cov = {}
    for edge in db.graph.edges(keys=True):
         cov[edge] = 0
    
    for r_id, mapping in mappings.items():
        if mapping is not None and mapping[-2]:
            for edge in mapping[-1]:
                cov[edge] += 1
    return cov

In [ ]:
edge_cov = get_edge_cov(mappings=mappings, db=dbs[400])

In [ ]:
for e_id, e_cov in edge_cov.items():
    if e_cov == 0:
        print(e_id, e_cov)

In [ ]:
edge_cov[(4554, 4728,0)]

In [ ]:
for r_id, mapping in mappings.items():
    if mapping is not None and mapping[-2]:
        if (4445, 10624, 0) in mapping[-1]:
            print(r_id, mapping[-1])

In [ ]:
monomers2units(long_edges[(5120, 11585, 0)]) # D in unit form

## Connection between E  = (5120, 11585, 0) and F = (2929, 1532, 0) 

In [ ]:
dbs[400].graph.get_edge_data(11585, 2929, 0)['edge_kmer']

In [ ]:
dbs[400].graph.get_edge_data(11585, 2929, 1)['edge_kmer']

In [ ]:
for i, (c1, c2) in enumerate(zip(dbs[400].graph.get_edge_data(11585, 2929, 0)['edge_kmer'],
                                 dbs[400].graph.get_edge_data(11585, 2929, 1)['edge_kmer'])):
    if c1 == c2 :
        print(i, c1.lower(), c2.lower())
    else:
        print(i, c1, c2, '!')

In [ ]:
monomers2units(long_edges[(2929, 2239, 0)]) # F in unit form

## Connection between F = (2929, 1532, 0) and G  = (2466, 5451, 0) 

In [ ]:
monomers2units(long_edges[(2466, 5451, 0)]) # G in unit form

In [ ]:
monomer_strings['bee10b9a-2130-48fe-ae17-ab10d9f1f4bf'][:374] == \
db.graph.get_edge_data(2929, 2239, 0)['edge_kmer'][-374:]

In [ ]:
monomer_strings['bee10b9a-2130-48fe-ae17-ab10d9f1f4bf'][:374] == \
db.graph.get_edge_data(3889, 2239, 0)['edge_kmer'][-374:]

In [ ]:
db.graph.get_edge_data(2929, 2239, 0)['edge_kmer'][-374:]

In [ ]:
mappings['bee10b9a-2130-48fe-ae17-ab10d9f1f4bf']

In [ ]:
mappings['a571367d-42ce-4bda-a218-0c38dc511dcc']

In [ ]:
monomer_strings['a571367d-42ce-4bda-a218-0c38dc511dcc']

In [ ]:
lrd_report.records['b988001c-6155-4fa8-8300-2ec01c12eeb1'].string

In [ ]:
mappings['6ee658cf-a26e-4c5a-87e1-aaa6a554d8a9']

In [ ]:
mappings['7bacbcf9-5c10-4e40-b09c-5b8c07ec0470']

In [ ]:
for r_id, mapping in mappings.items():
    if mapping is not None and mapping[-2]:
        if (2929, 2239, 0) in mapping[-1] and len(mapping[-1]) > 1:
            print(r_id, mapping[-1])

In [ ]:
for r_id, mapping in mappings.items():
    if mapping is not None and mapping[-2]:
        if (2317, 5625, 0) in mapping[-1] and len(mapping[-1]) > 1:
            print(r_id, mapping[-1])

In [ ]:
for r_id, mapping in mappings.items():
    if mapping is not None and mapping[-2]:
        if (2415, 4554, 0) in mapping[-1] and len(mapping[-1]) > 1:
            print(r_id, mapping[-1])

In [ ]:
for r_id, mapping in mappings.items():
    if mapping is not None and mapping[-2]:
        if (4728, 3889, 0) in mapping[-1] and len(mapping[-1]) > 10:
            print(r_id, mapping[-1])

In [ ]:
for r_id, mapping in mappings.items():
    if mapping is not None and mapping[-2]:
        if (4728, 3889, 1) in mapping[-1] and len(mapping[-1]) > 1:
            print(r_id, mapping[-1])

In [ ]:
for r_id, mapping in mappings.items():
    if mapping is not None and mapping[-2]:
        if (2466, 4554, 0) in mapping[-1] and len(mapping[-1]) > 5:
            print(r_id, mapping[-1])

In [ ]:
for r_id, mapping in mappings.items():
    if mapping is not None and mapping[-2]:
        if (4728, 3889, 1) in mapping[-1]:
            print(r_id, mapping[-1])

## Connection between G = (2466, 5451, 0) and H = (5509, 11189, 0) 

In [ ]:
mappings['0b1f944b-90f7-4e17-a1cd-f02d38d0f84c']

## Connection between H = (5509, 11189, 0) and * = (11189, 7050, 0)

In [ ]:
mappings['e292a036-e0a2-458b-8ee2-8e61272844a3']

In [ ]:
for r_id, mapping in mappings.items():
    if mapping is not None and mapping[-2]:
        if (11189, 7050, 0) in mapping[-1]:
            print(r_id, mapping[-1])

In [ ]:
monomers2units(long_edges[(11189, 7050, 0)]) # * in unit form

In [ ]:
monomers2units(long_edges[(5509, 11189, 0)]) # H in unit form

# Manual scaffold guess

In [ ]:
def gﬁfold_path += mappings['6ee658cf-a26e-4c5a-87e1-aaa6a554d8a9'][-1] # from E inside dublication
    scaffold_path += [(2415, 4554, 0), (4554, 4728, 0), (4728, 3889, 1), (3889, 2239, 0)]
    scaffold_path += mappings['bee10b9a-2130-48fe-ae17-ab10d9f1f4bf'][-1] # up to G
    scaffold_path += mappings['0b1f944b-90f7-4e17-a1cd-f02d38d0f84c'][-1][1:] # up to H
    scaffold_path += mappings['e292a036-e0a2-458b-8ee2-8e61272844a3'][-1][1:]
    
    scaffold = db.get_path(scaffold_path)
    return scaffold, scaffold_path

In [ ]:
scaffold, scaffold_path = get_scaffold_v1_Nov18(dbs[400])

In [ ]:
edlib.align(scaffold, ref_monomers)

In [ ]:
cov = get_cov(ref_monomers, monomer_strings)
plt.plot(cov)
plt.title('Coverage of Karen cen6 assembly')

In [ ]:
cov = get_cov(scaffold, monomer_strings)
plt.plot(cov)
plt.title('Coverage of our cen6 assembly')

In [ ]:
monomer_strings['ab79a298-1f1a-44da-a174-6a52ae8abfcd']

In [ ]:
lrd_report.records['ab79a298-1f1a-44da-a174-6a52ae8abfcd'].string[18:][1268:1274]

In [ ]:
len(lrd_report.records['ab79a298-1f1a-44da-a174-6a52ae8abfcd'].string)

In [ ]:
Counter(monomer_strings['ab79a298-1f1a-44da-a174-6a52ae8abfcd'])['=']